In [51]:
import torch
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [52]:
num_to_label= {
    0: "backpack",
    1: "book",
    2: "car",
    3: "pizza",
    4: "sandwich",
    5: "snake",
    6: "sock",
    7: "tiger",
    8: "tree",
    9: "watermelon"
}

label_to_num = dict((v,k) for k,v in num_to_label.items())

def obtain_labels(root_dir):
    labels = []

    ok = os.walk(root_dir, topdown=False)
    for root, _, files in ok:
        if root_dir == root: continue
        root = os.path.basename(root)
        labels += list(map(lambda x: [x, root], files))

    return np.array(labels)

In [53]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = obtain_labels(img_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx, 1], self.img_labels[idx, 0])
        image = read_image(img_path)
        label = self.img_labels[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
dataset_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

class LabelToNum(torch.nn.Module):
    def forward(self, label):
        return label_to_num[label]

label_transform = LabelToNum()

In [54]:
rn34_model = models.resnet34()
num_ftrs = rn34_model.fc.in_features
rn34_model.fc = torch.nn.Linear(num_ftrs, 10)

In [55]:
validation_split = 0.2

## Obtain Real datasets and split for validation sets.
train_dataset_real = CustomImageDataset(img_dir="data/real_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_real = CustomImageDataset(img_dir="data/real_test", transform=dataset_transform, target_transform=label_transform)

val_size = int(validation_split*len(train_dataset_real))
train_size = len(train_dataset_real)-val_size
train_subset_real, val_subset_real = random_split(train_dataset_real, [train_size, val_size])

## Obtain Sketch datasets and split for validation sets.
train_dataset_sketch = CustomImageDataset(img_dir="data/sketch_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_sketch = CustomImageDataset(img_dir="data/sketch_test", transform=dataset_transform, target_transform=label_transform)

val_size = int(validation_split*len(train_dataset_sketch))
train_size = len(train_dataset_sketch)-val_size
train_subset_sketch, val_subset_sketch = random_split(train_dataset_sketch, [train_size, val_size])

In [56]:
print(train_dataset_real.img_labels[0, 1])
img_path = os.path.join(train_dataset_real.img_dir, train_dataset_real.img_labels[0, 1], train_dataset_real.img_labels[0, 0])

backpack


In [57]:
def train(src_loader, model, loss_fn, optimizer, epoch, device):
    running_loss = 0
    train_loss = 0
   
    for idx, (data, targets) in enumerate(src_loader):

        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = loss_fn(scores, targets)
        
        
        optimizer.zero_grad()
        loss.backward()

        # update the parameters according to gradients
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        train_loss += loss.item()

        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 25))
            running_loss = 0.0
    
    train_loss = train_loss/(idx+1)

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    
    return train_loss


def test(src_loader, model, loss_fn, device):

    valid_loss = 0
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for data, targets in src_loader:
            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)
            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

            valid_loss += loss_fn(scores, targets)
        
        print(f'{num_correct}/{num_samples} accuracy {num_correct/num_samples}')

    # Gather data and report
    valid_loss /= len(src_loader)
    accuracy = num_correct/num_samples
    print("Test Error: \n   Accuracy: {:.2f}, Avg loss: {:.4f} \n".format(100*accuracy, valid_loss))
    
    return valid_loss, accuracy

In [58]:
from torch.utils.tensorboard import SummaryWriter

In [59]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Training init
learning_rate = 0.001
epochs = 50
batch_size = 128

In [60]:
# Real DataLoaders
dl_train_real = DataLoader(
    train_subset_real,
    batch_size=batch_size,
    shuffle=True,
)

dl_valid_real = DataLoader(
    val_subset_real,
    batch_size=batch_size,
    shuffle=True,
)

dl_test_real = DataLoader(
    test_dataset_real,
    batch_size=batch_size,
    shuffle=True,
)

## Sketch DataLoaders
dl_train_sketch = DataLoader(
    train_subset_sketch,
    batch_size=batch_size,
    shuffle=True,
)

dl_valid_sketch = DataLoader(
    val_subset_sketch,
    batch_size=batch_size,
    shuffle=True
)

dl_test_sketch = DataLoader(
    test_dataset_sketch,
    batch_size=batch_size,
    shuffle=True,
)

In [61]:
print(device)

cpu


In [62]:
## Optimizer construction using Adam.
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(
    params,
    lr=learning_rate,
)
rn34_model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss = train(dl_train_sketch, rn34_model, loss_fn, optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

KeyboardInterrupt: 

In [ ]:
loss_weight = 10
epochs = 20
learning_rate = 0.001

In [ ]:
class CORAL(torch.nn.Module):
    def forward(self, source, target, device):
        d = source.size(1)
        source_cov = self.covariance_matrix(source, device)
        target_cov = self.covariance_matrix(target, device)

        loss = torch.square(torch.linalg.matrix_norm(source_cov-target_cov, ord='fro'))
        loss /= 4*(d**2)
        return loss

    def covariance_matrix(self, x, device):
        ns = x.size(0)
        ones_vec = torch.ones(ns).reshape(ns, 1).to(device)
        meow = torch.mm(ones_vec.t(), x)
        cov = torch.mm(x.t(), x) - (1/ns)*torch.mm(meow.t(), meow)
        cov /= (ns-1)
        return cov

In [ ]:
def train_coral(src_loader, tgt_loader, model, loss_fn, optimizer, epoch, device):
    running_loss = 0
    train_loss = 0

    coral_loss = CORAL()
   
    for idx, (src_data, tgt_data) in enumerate(zip(src_loader, tgt_loader)):
        print(idx)

        src_input, src_label = src_data
        tgt_input, _ = tgt_data

        src_input = src_input.to(device)
        src_label = src_label.to(device)
        tgt_input = tgt_input.to(device)

        src_outputs = model(src_input)
        tgt_outputs = model(tgt_input)
        
        loss = loss_fn(src_outputs, src_label)
        loss_coral = coral_loss(src_outputs, tgt_outputs, device)
        loss += loss_weight * loss_coral
        
        optimizer.zero_grad()
        loss.backward()

        # update the parameters according to gradients
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        train_loss += loss.item()

        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 100))
            running_loss = 0.0
    
    train_loss = train_loss/(idx+1)

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    
    return train_loss

In [ ]:
rn34_model = models.resnet34()
num_ftrs = rn34_model.fc.in_features
rn34_model.fc = torch.nn.Linear(num_ftrs, 10)

In [ ]:
## Optimizer construction using SGD.
params = [p for p in rn34_model.parameters() if p.requires_grad]
sgd_optimizer = torch.optim.SGD(
    params,
    lr=learning_rate,
)
rn34_model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss = train_coral(dl_train_real, dl_train_sketch, rn34_model, loss_fn, sgd_optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

0
1
2
3
4
5
6
Epoch [1], Train Loss: 2.2551, 
19/391 accuracy 0.04859334975481033
Test Error: 
   Accuracy: 4.86, Avg loss: 2.3873 

Epoch: 1 Train_loss: 2.25508781841823 Valid_loss: 2.3873109817504883 Valid_acc: 0.04859334975481033
0
1
2
3
4
5
6
Epoch [2], Train Loss: 2.2418, 
19/391 accuracy 0.04859334975481033
Test Error: 
   Accuracy: 4.86, Avg loss: 2.3933 

Epoch: 2 Train_loss: 2.2417832783290317 Valid_loss: 2.39327335357666 Valid_acc: 0.04859334975481033
0
1
2
3
4
5
6
Epoch [3], Train Loss: 2.2112, 
19/391 accuracy 0.04859334975481033
Test Error: 
   Accuracy: 4.86, Avg loss: 2.4544 

Epoch: 3 Train_loss: 2.2111683913639615 Valid_loss: 2.454374074935913 Valid_acc: 0.04859334975481033
0


KeyboardInterrupt: 